In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import os
from pathlib import Path

In [ ]:
# --- Configuration ---
TICKERS = ["INFY.NS", "HDFCBANK.NS", "TATAMOTORS.NS", "ASIANPAINT.NS", "WIPRO.NS"]   # <-- edit tickers
START = "2023-01-01"                                          # <-- edit start date
END   = None                                                 # None = up to latest date

# --- Setup output folder in the Colab environment ---
# This creates a folder named 'data' in the Colab file browser (left panel)
out_dir = Path("data")
out_dir.mkdir(parents=True, exist_ok=True)
print(f"Output directory '{out_dir}' is ready.")

def download_single_ticker(ticker: str, start=START, end=END):
    """
    Download a single ticker from Yahoo Finance and return
    OHLCV-only DataFrame with lowercase columns and 'datetime' index.
    Handles MultiIndex columns from yfinance.
    """
    print(f"Downloading {ticker}...")
    df = yf.download(ticker, start=start, end=end, auto_adjust=False, progress=False)
    if df.empty:
        print(f"[Warning] No data found for {ticker}")
        return df

    # If MultiIndex columns (e.g., ('Open', 'RELIANCE.NS')), take first level
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)

    # Keep only standard OHLCV columns
    keep = [c for c in ["Open", "High", "Low", "Close", "Volume"] if c in df.columns]
    df = df[keep].copy()

    # Lowercase column names and set index name
    df.columns = [str(c).lower() for c in df.columns]
    df.index.name = "datetime"
    return df

# --- Main Loop to Download and Save ---
for t in TICKERS:
    df = download_single_ticker(t)
    if df.empty:
        continue

    # Create a clean filename (e.g., RELIANCE.NS -> RELIANCE_NS.csv)
    out_path = out_dir / f"{t.replace('.', '_')}.csv"
    df.to_csv(out_path, index=True, index_label="datetime")
    print(f"[OK] Saved {t} -> {out_path}\n")

print("All tickers processed!")

Output directory 'data' is ready.
[OK] Saved INFY.NS -> data/INFY_NS.csv

[OK] Saved HDFCBANK.NS -> data/HDFCBANK_NS.csv

[OK] Saved TATAMOTORS.NS -> data/TATAMOTORS_NS.csv

[OK] Saved ASIANPAINT.NS -> data/ASIANPAINT_NS.csv

[OK] Saved WIPRO.NS -> data/WIPRO_NS.csv

All tickers processed!


In [ ]:
class BacktestResults:
    def __init__(self, df, cash, commission):
        self.df = df
        self.cash = cash
        self.commission = commission
        self.portfolio = [self.cash]
        self.pnl = []
        self.pnl_up = []
        self.pnl_down = []
        self.pnl_net = [0]
        self.num_trades = 0
        self.long_trades = 0
        self.short_trades = 0
        self.fee_static = 0
        self.fee_comp = 0
        self.entries = {}
        self.exits = {}
        self.position = None
        self.entry = 0
        self.shares_comp = 0
        self.shares_static = 0
        for i in range(len(self.df)-1):
            if self.portfolio[-1] <= 0: self.portfolio[-1] = 0; break
            if self.df['signals'][i] == 1:
                if self.position is None:
                    self.entry = self.df['open'][i+1]
                    # Choose how much of the portfolio to allocate each trade
                    # Choose how much of the portfolio to allocate each trade
                    risk_fraction = 1

                       # 1.0 = 100% portfolio, 0.5 = 50%, 1.5 = 150% (leverage)
                    self.shares_comp = (self.portfolio[-1] * risk_fraction) / self.entry


                    self.shares_static = self.cash / self.entry
                    self.position = 'long'; self.entries[i] = self.entry
                elif self.position == 'short':
                    exit_price = self.df['open'][i+1]
                    pnl_comp = self.shares_comp * (self.entry - exit_price)
                    self.portfolio.append(self.portfolio[-1] + pnl_comp - (self.commission * self.shares_comp * (self.entry + exit_price)))
                    pnl_static = self.shares_static * (self.entry - exit_price) - (self.commission * self.shares_static * (self.entry + exit_price))
                    self.pnl.append(pnl_static); self.pnl_net.append(self.pnl_net[-1] + pnl_static)
                    if pnl_static > 0: self.pnl_up.append(pnl_static)
                    else: self.pnl_down.append(pnl_static)
                    self.num_trades += 1; self.short_trades += 1
                    self.fee_static += (self.commission * self.shares_static * (self.entry + exit_price))
                    self.fee_comp += (self.commission * self.shares_comp * (self.entry + exit_price))
                    self.position = None; self.exits[i] = exit_price
            elif self.df['signals'][i] == -1:
                if self.position == 'long':
                    exit_price = self.df['open'][i+1]
                    pnl_comp = self.shares_comp * (exit_price - self.entry)
                    self.portfolio.append(self.portfolio[-1] + pnl_comp - (self.commission * self.shares_comp * (self.entry + exit_price)))
                    pnl_static = self.shares_static * (exit_price - self.entry) - (self.commission * self.shares_static * (self.entry + exit_price))
                    self.pnl.append(pnl_static); self.pnl_net.append(self.pnl_net[-1] + pnl_static)
                    if pnl_static > 0: self.pnl_up.append(pnl_static)
                    else: self.pnl_down.append(pnl_static)
                    self.num_trades += 1; self.long_trades += 1
                    self.fee_static += (self.commission * self.shares_static * (self.entry + exit_price))
                    self.fee_comp += (self.commission * self.shares_comp * (self.entry + exit_price))
                    self.position = None; self.exits[i] = exit_price
                elif self.position is None:
                    self.entry = self.df['open'][i+1]
                    # Choose how much of the portfolio to allocate each trade
                    risk_fraction = 1   # 1.0 = 100% portfolio, 0.5 = 50%, 1.5 = 150% (leverage)



                    self.shares_comp = (self.portfolio[-1] * risk_fraction) / self.entry

                    self.shares_static = self.cash / self.entry
                    self.position = 'short'; self.entries[i] = self.entry
        if self.position is not None:
            if self.position == 'long': self.df.loc[len(self.df)-1, 'signals'] = -1
            elif self.position == 'short': self.df.loc[len(self.df)-1, 'signals'] = 1
            self.__init__(self.df, self.cash, self.commission)

    def trade_history(self):
        print("--- TRADE HISTORY ---\n")
        position = None
        for i in range(len(self.df)-1):
            if self.df['signals'][i] == 1:
                if position is None:
                    print("LONG POSITION")
                    print(f"{self.df['datetime'][i+1]}: ENTRY - BUY  @ {self.df['open'][i+1]:.2f}")
                    position = 'long'
                elif position == 'short':
                    print(f"{self.df['datetime'][i+1]}: EXIT  - BUY  @ {self.df['open'][i+1]:.2f}\n")
                    position = None
            elif self.df['signals'][i] == -1:
                if position == 'long':
                    print(f"{self.df['datetime'][i+1]}: EXIT  - SELL @ {self.df['open'][i+1]:.2f}\n")
                    position = None
                elif position is None:
                    print("SHORT POSITION")
                    print(f"{self.df['datetime'][i+1]}: ENTRY - SELL @ {self.df['open'][i+1]:.2f}")
                    position = 'short'

    # --- TRADE DURATION AND STATS ---
    def trade_metrics(self):
        """Prints statistics related to trade count, win rate, and duration."""
        print("\n--- TRADE ANALYSIS ---")
        if self.num_trades == 0:
            print("No trades were executed.")
            return

        # Calculate trade durations in terms of number of bars
        entry_indices = sorted(list(self.entries.keys()))
        exit_indices = sorted(list(self.exits.keys()))

        # Ensure we have matching entry/exit pairs before calculating
        if len(entry_indices) == len(exit_indices):
            trade_durations = [exit_indices[i] - entry_indices[i] for i in range(len(entry_indices))]
            avg_duration = np.mean(trade_durations)
            max_duration = np.max(trade_durations)
            print(f"Average Trade Duration: {avg_duration:.2f} bars")
            print(f"Maximum Trade Duration: {max_duration} bars")
        else:
            print("Warning: Mismatch in entries/exits, cannot calculate duration.")

        print(f"Total Trades: {self.num_trades}")
        print(f"Winning Trades: {len(self.pnl_up)}")
        print(f"Losing Trades: {len(self.pnl_down)}")
        print(f"Win Rate: {len(self.pnl_up) / self.num_trades:.2%}")
        print(f"Number of Long Trades: {self.long_trades}")
        print(f"Number of Short Trades: {self.short_trades}")

    def static_metrics(self):
        print("\n--- STATIC (FIXED CAPITAL) METRICS ---")
        print(f"Capital per Trade: {self.cash:.2f}")
        if len(self.pnl_net) > 1: print(f"Net Profit: {self.pnl_net[-1]:.2f}")
        else: print("Net Profit: 0.00")
        print(f"Total Transaction Fees: {self.fee_static:.2f}")

    def compounded_metrics(self):
        print("\n--- COMPOUNDED METRICS ---")
        if not self.portfolio or self.portfolio[0] == 0:
            print("No trades or portfolio empty."); return
        net_return = (self.portfolio[-1] / self.portfolio[0]) - 1
        num_years = (self.df['datetime'].iloc[-1] - self.df['datetime'].iloc[0]).days / 365.25
        bnh_return = self.df['close'].iloc[-1] / self.df['close'].iloc[0] - 1
        annual_return = (1 + net_return) ** (1 / num_years) - 1 if num_years > 0 else 0
        portfolio_arr = np.array(self.portfolio); running_max = np.maximum.accumulate(portfolio_arr)
        drawdown = (running_max - portfolio_arr) / running_max; max_draw = np.max(drawdown)
        sharpe_ratio, sortino_ratio = 0, 0
        if self.num_trades > 1 and num_years > 0:
            trade_returns = pd.Series(self.portfolio).pct_change().dropna()
            if trade_returns.std() != 0: sharpe_ratio = trade_returns.mean() / trade_returns.std() * np.sqrt(self.num_trades / num_years)
            negative_returns = trade_returns[trade_returns < 0]; downside_std = negative_returns.std()
            if downside_std != 0: sortino_ratio = trade_returns.mean() / downside_std * np.sqrt(self.num_trades / num_years)
        print(f"Initial Balance: {self.portfolio[0]:.2f}"); print(f"Final Balance: {self.portfolio[-1]:.2f}")
        print(f"Net Return: {net_return:.2%}"); print(f"Annualised Return: {annual_return:.2%}")
        print(f"Buy and Hold Return: {bnh_return:.2%}"); print(f"Maximum Drawdown: {max_draw:.2%}")
        print(f"Sharpe Ratio (annualised): {sharpe_ratio:.3f}"); print(f"Sortino Ratio (annualised): {sortino_ratio:.3f}")
        print(f"Total Transaction Fees: {self.fee_comp:.2f}")

    def plot_equity_and_drawdown(self):
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=("Equity Curves", "Portfolio Drawdown"))
        fig.add_trace(go.Scatter(x=list(range(len(self.portfolio))), y=self.portfolio, mode='lines', name="Compounded Equity", line=dict(color='cyan')), row=1, col=1)
        fig.add_trace(go.Scatter(x=list(range(len(self.pnl_net))), y=[self.cash + pnl for pnl in self.pnl_net], mode='lines', name="Static Equity", line=dict(color='orange', dash='dash')), row=1, col=1)
        portfolio_arr = np.array(self.portfolio); running_max = np.maximum.accumulate(portfolio_arr)
        drawdown = (running_max - portfolio_arr) / running_max * 100
        fig.add_trace(go.Scatter(x=list(range(len(drawdown))), y=drawdown, mode='lines', name="Drawdown", fill='tozeroy', line=dict(color='red', width=1)), row=2, col=1)
        fig.update_layout(title="Performance Summary: Equity and Drawdown", height=800, template="plotly_dark", legend_title="Legend")
        fig.update_xaxes(title_text="# of Trades", row=2, col=1); fig.update_yaxes(title_text="Portfolio Value ($)", row=1, col=1); fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)
        fig.show()

In [ ]:
class Indicators:
    @staticmethod
    def add_rsi(df, period: int = 14, price_col: str = "close", out_col: str = "rsi"):
        prices = df[price_col].to_numpy(dtype=float)
        n = len(prices)
        rsi = np.full(n, np.nan, dtype=float)
        if n > period:
            deltas = np.diff(prices)
            gains = np.clip(deltas, a_min=0.0, a_max=None)
            losses = np.clip(-deltas, a_min=0.0, a_max=None)
            avg_gain = np.full(n, np.nan, dtype=float)
            avg_loss = np.full(n, np.nan, dtype=float)
            avg_gain[period] = gains[:period].mean()
            avg_loss[period] = losses[:period].mean()
            for i in range(period + 1, n):
                gain = gains[i - 1]
                loss = losses[i - 1]
                avg_gain[i] = (avg_gain[i - 1] * (period - 1) + gain) / period
                avg_loss[i] = (avg_loss[i - 1] * (period - 1) + loss) / period
            rs = np.divide(avg_gain, avg_loss, out=np.full(n, np.nan, dtype=float), where=avg_loss != 0)
            rsi_vals = 100.0 - (100.0 / (1.0 + rs))
            rsi = rsi_vals
            rsi[:period] = np.nan
        df[out_col] = rsi
        return df

    @staticmethod
    def _ema(values: np.ndarray, period: int) -> np.ndarray:
        n = len(values)
        ema = np.full(n, np.nan, dtype=float)
        if n == 0 or period <= 0:
            return ema
        if n >= period:
            seed_idx = period - 1
            seed = np.mean(values[:period])
        else:
            seed_idx = 0
            seed = np.mean(values[: max(1, n)])
        ema[seed_idx] = seed
        alpha = 2.0 / (period + 1.0)
        prev = seed
        for i in range(seed_idx + 1, n):
            x = values[i]
            prev = (x - prev) * alpha + prev
            ema[i] = prev
        return ema

    @staticmethod
    def add_macd(
        df, fast: int = 12, slow: int = 26, signal: int = 9,
        price_col: str = "close", macd_col: str = "macd",
        signal_col: str = "macd_signal", hist_col: str = "macd_hist",
    ):
        prices = df[price_col].to_numpy(dtype=float)
        ema_fast = Indicators._ema(prices, fast)
        ema_slow = Indicators._ema(prices, slow)
        macd = ema_fast - ema_slow
        signal_line = Indicators._ema(macd, signal)
        hist = macd - signal_line
        df[macd_col] = macd
        df[signal_col] = signal_line
        df[hist_col] = hist
        return df

    # --- New Indicators ---
    @staticmethod
    def add_sma(df, period: int = 20, price_col: str = "close", out_col: str = "sma"):
        df[out_col] = df[price_col].rolling(window=period).mean()
        return df

    @staticmethod
    def add_bollinger_bands(df, period: int = 20, price_col: str = "close",
                            upper_col: str = "bb_upper", lower_col: str = "bb_lower", mid_col: str = "bb_mid"):
        sma = df[price_col].rolling(window=period).mean()
        std = df[price_col].rolling(window=period).std()
        df[mid_col] = sma
        df[upper_col] = sma + (2 * std)
        df[lower_col] = sma - (2 * std)
        return df

    @staticmethod
    def add_atr(df, period: int = 14, high_col: str = "high", low_col: str = "low", close_col: str = "close", out_col: str = "atr"):
        high = df[high_col].to_numpy(dtype=float)
        low = df[low_col].to_numpy(dtype=float)
        close = df[close_col].to_numpy(dtype=float)

        tr = np.zeros(len(df))
        tr[1:] = np.maximum.reduce([
            high[1:] - low[1:],
            np.abs(high[1:] - close[:-1]),
            np.abs(low[1:] - close[:-1])
        ])
        atr = np.full(len(df), np.nan, dtype=float)
        atr[period] = tr[1:period+1].mean()
        for i in range(period+1, len(df)):
            atr[i] = (atr[i-1] * (period - 1) + tr[i]) / period
        df[out_col] = atr
        return df

    @staticmethod
    def add_stochastic(df, k_period: int = 14, d_period: int = 3,
                       high_col: str = "high", low_col: str = "low", close_col: str = "close",
                       k_col: str = "stoch_k", d_col: str = "stoch_d"):
        low_min = df[low_col].rolling(window=k_period).min()
        high_max = df[high_col].rolling(window=k_period).max()
        stoch_k = 100 * (df[close_col] - low_min) / (high_max - low_min)
        stoch_d = stoch_k.rolling(window=d_period).mean()
        df[k_col] = stoch_k
        df[d_col] = stoch_d
        return df


In [ ]:
def plot_strategy_on_chart(df: pd.DataFrame):
    """
    Plots the price action, RSI, and all trade signals on a multi-panel chart.

    Args:
        df (pd.DataFrame): DataFrame must contain ohlc, datetime, rsi, and signals columns.
    """
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=("Price Action & Trades", "RSI Indicator"),
        row_heights=[0.7, 0.3]
    )

    # --- Panel 1: Price and Trade Markers ---
    fig.add_trace(go.Candlestick(
        x=df['datetime'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name="Price"
    ), row=1, col=1)

    # Find the locations of all buy and sell signals
    # A "buy" action is either entering a long or exiting a short (signal = 1)
    # A "sell" action is either exiting a long or entering a short (signal = -1)
    buy_signals = df[df['signals'] == 1]
    sell_signals = df[df['signals'] == -1]

    # The trade occurs at the 'open' of the NEXT bar (i+1)
    # We get the date and price for the markers from the next bar
    buy_marker_dates = [df.loc[i + 1, 'datetime'] for i in buy_signals.index if i + 1 < len(df)]
    buy_marker_prices = [df.loc[i + 1, 'open'] for i in buy_signals.index if i + 1 < len(df)]

    sell_marker_dates = [df.loc[i + 1, 'datetime'] for i in sell_signals.index if i + 1 < len(df)]
    sell_marker_prices = [df.loc[i + 1, 'open'] for i in sell_signals.index if i + 1 < len(df)]

    # Add buy markers to the price chart
    fig.add_trace(go.Scatter(
        x=buy_marker_dates,
        y=buy_marker_prices,
        mode='markers',
        marker=dict(color='lime', symbol='triangle-up', size=12, line=dict(width=1, color='black')),
        name='Buy/Cover Action'
    ), row=1, col=1)

    # Add sell markers to the price chart
    fig.add_trace(go.Scatter(
        x=sell_marker_dates,
        y=sell_marker_prices,
        mode='markers',
        marker=dict(color='red', symbol='triangle-down', size=12, line=dict(width=1, color='black')),
        name='Sell/Short Action'
    ), row=1, col=1)


    # --- Panel 2: RSI ---
    fig.add_trace(go.Scatter(
        x=df['datetime'], y=df['rsi'],
        mode='lines', name='RSI', line=dict(color='cyan', width=1.5)
    ), row=2, col=1)
    # Add RSI overbought/oversold horizontal lines
    fig.add_hline(y=65, line_dash="dash", line_color="red", line_width=1, row=2, col=1)
    fig.add_hline(y=35, line_dash="dash", line_color="lime", line_width=1, row=2, col=1)


    # --- Layout and Formatting ---
    fig.update_layout(
        title="Strategy Visualization: Trades on Price Chart with RSI",
        height=800,
        template="plotly_dark",
        legend_title="Legend",
        xaxis_rangeslider_visible=False,  # Hide the default range slider
    )
    fig.update_yaxes(title_text="Price (INR)", row=1, col=1)
    fig.update_yaxes(title_text="RSI Value", row=2, col=1)

    fig.show()

In [ ]:
# Load CSV
csv_path = "data/HDFCBANK_NS.csv" # Make sure this file is in the same directory
df = pd.read_csv(csv_path, parse_dates=["datetime"])
df.columns = df.columns.str.lower()
df = df.sort_values("datetime").reset_index(drop=True)
# Map df -> px for strategies
px = df.rename(columns={"datetime": "Date"})[["Date", "open", "high", "low", "close"]].copy()
px.columns = ["Date", "Open", "High", "Low", "Close"]

# Using only the last 3 years for a clearer plot
df = df[df['datetime'] > '2023-01-01'].reset_index(drop=True)

# Add indicators
df = Indicators.add_rsi(df, period=14, price_col="close", out_col="rsi")
# df = Indicators.add_macd(df, fast=12, slow=26, signal=9,
#                          price_col="close",
#                          macd_col="macd", signal_col="macd_signal", hist_col="macd_hist")

In [ ]:
# ===========================================================
# Strategy Type:
#   Hybrid Mean-Reversion Strategy using RSI + Bollinger Bands
#
#   - Indicators used:
#       1. RSI (Relative Strength Index)
#       2. Bollinger Bands (20-period, ±3 std dev)
#
#   - Entry Logic:
#       * Long entry when RSI < 40 OR price < lower Bollinger Band
#       * Short entry when RSI > 70 OR price > upper Bollinger Band
#
#   - Exit Logic:
#       * Long exit when price ≥ upper band OR RSI > 70 OR trailing stop hit
#       * Short exit when price ≤ lower band OR RSI < 40 OR trailing stop hit
#
#   - Risk Management:
#       * Mandatory 10% Trailing Stop Loss (locks profits, cuts losses)
#
#   - Positioning:
#       * Signals column is set to 1 for long, -1 for short, 0 for flat
# ===========================================================

window = 20
num_std = 3
trail_pct = 0.10  # 10% trailing stop loss

# --- Compute Bollinger Bands ---
df["middle_band"] = df["close"].rolling(window).mean()
df["std_dev"] = df["close"].rolling(window).std()
df["upper_band"] = df["middle_band"] + num_std * df["std_dev"]
df["lower_band"] = df["middle_band"] - num_std * df["std_dev"]

# --- Initialize strategy state variables ---
df["signals"] = 0           # trading signal: 1 = long, -1 = short, 0 = no action
position = None             # current market position: None, "long", "short"
entry_price = None          # entry price for current position
trailing_stop = None        # trailing stop price

# --- Iterate over price series ---
for i in range(len(df)):
    price = df.loc[i, "close"]
    rsi_val = df.loc[i, "rsi"]
    upper = df.loc[i, "upper_band"]
    lower = df.loc[i, "lower_band"]

    # ENTRY RULES (only when not in a trade)
    if position is None:
        # Long entry: RSI < 40 OR price below lower band
        if rsi_val < 40 or price < lower:
            df.loc[i, "signals"] = 1
            position = "long"
            entry_price = price
            trailing_stop = entry_price * (1 - trail_pct)  # initial stop below entry

        # Short entry: RSI > 70 OR price above upper band
        elif rsi_val > 70 or price > upper:
            df.loc[i, "signals"] = -1
            position = "short"
            entry_price = price
            trailing_stop = entry_price * (1 + trail_pct)  # initial stop above entry

    # EXIT RULES (if already long)
    elif position == "long":
        # Update trailing stop upwards when price rises
        trailing_stop = max(trailing_stop, price * (1 - trail_pct))

        # Exit long if: price touches upper band OR RSI > 70 OR stop loss hit
        if price >= upper or rsi_val > 70 or price <= trailing_stop:
            df.loc[i, "signals"] = -1
            position = None
            entry_price = None
            trailing_stop = None

    # EXIT RULES (if already short)
    elif position == "short":
        # Update trailing stop downwards when price falls
        trailing_stop = min(trailing_stop, price * (1 + trail_pct))

        # Exit short if: price touches lower band OR RSI < 40 OR stop loss hit
        if rsi_val < 40 or price <= lower or price >= trailing_stop:
            df.loc[i, "signals"] = 1
            position = None
            entry_price = None
            trailing_stop = None

# ✅ Force close on the penultimate bar if still in a trade
if position is not None:
    df.loc[df.index[-2], "signals"] = -1 if position == "long" else 1


In [ ]:
# --- Plot the new strategy visualization ---
print("\nGenerating strategy visualization chart...")
plot_strategy_on_chart(df)


Generating strategy visualization chart...


In [ ]:
# --- Run Backtest and Display Full Report ---
INITIAL_CASH = 100000
COMMISSION_RATE = 0.001

results = BacktestResults(df, INITIAL_CASH, COMMISSION_RATE)

In [ ]:
# Call the new functions to print the full report
results.trade_history()

--- TRADE HISTORY ---

LONG POSITION
2023-02-24 00:00:00: ENTRY - BUY  @ 808.95
2023-04-17 00:00:00: EXIT  - SELL @ 860.00

SHORT POSITION
2023-05-05 00:00:00: ENTRY - SELL @ 819.50
2023-05-26 00:00:00: EXIT  - BUY  @ 809.50

LONG POSITION
2023-06-02 00:00:00: ENTRY - BUY  @ 806.22
2023-07-03 00:00:00: EXIT  - SELL @ 856.25

SHORT POSITION
2023-07-04 00:00:00: ENTRY - SELL @ 861.72
2023-08-14 00:00:00: EXIT  - BUY  @ 805.50

LONG POSITION
2023-08-16 00:00:00: ENTRY - BUY  @ 791.55
2023-10-27 00:00:00: EXIT  - SELL @ 734.75

LONG POSITION
2023-10-30 00:00:00: ENTRY - BUY  @ 731.62
2023-12-05 00:00:00: EXIT  - SELL @ 818.25

SHORT POSITION
2023-12-06 00:00:00: ENTRY - SELL @ 819.22
2024-01-18 00:00:00: EXIT  - BUY  @ 747.00

LONG POSITION
2024-01-19 00:00:00: ENTRY - BUY  @ 752.97
2024-04-05 00:00:00: EXIT  - SELL @ 769.50

SHORT POSITION
2024-04-08 00:00:00: ENTRY - SELL @ 777.47
2024-05-10 00:00:00: EXIT  - BUY  @ 719.50

LONG POSITION
2024-05-13 00:00:00: ENTRY - BUY  @ 716.00
2024-06

In [ ]:
results.static_metrics()
results.compounded_metrics()
results.trade_metrics()


--- STATIC (FIXED CAPITAL) METRICS ---
Capital per Trade: 100000.00
Net Profit: 74651.33
Total Transaction Fees: 3428.62

--- COMPOUNDED METRICS ---
Initial Balance: 100000.00
Final Balance: 200141.65
Net Return: 100.14%
Annualised Return: 29.65%
Buy and Hold Return: 18.04%
Maximum Drawdown: 10.06%
Sharpe Ratio (annualised): 1.583
Sortino Ratio (annualised): 3.582
Total Transaction Fees: 5163.21

--- TRADE ANALYSIS ---
Average Trade Duration: 29.53 bars
Maximum Trade Duration: 66 bars
Total Trades: 17
Winning Trades: 13
Losing Trades: 4
Win Rate: 76.47%
Number of Long Trades: 9
Number of Short Trades: 8


In [ ]:
# Display the new plot
results.plot_equity_and_drawdown()